# Quickstart: Using scDataset with the Tahoe-100M Dataset
This tutorial demonstrates how to use scDataset to load the  Tahoe-100M single-cell dataset in h5ad format and train a simple linear classifier in PyTorch.

## Downloading the Tahoe-100M h5ad Files
You can download the Tahoe-100M dataset in h5ad format from the [Arc Institute Google Cloud Storage](https://github.com/ArcInstitute/arc-virtual-cell-atlas/blob/main/tahoe-100M/README.md) or convert it from the [HuggingFace version](https://huggingface.co/datasets/tahoebio/Tahoe-100M). For this tutorial, download one or more `plate*_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad` files and note their paths.

In [ ]:
# Install required packages (uncomment if running in a fresh environment)
# %pip install scipy scikit-learn tqdm torch anndata scDataset 

# Import libraries
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import anndata as ad
from anndata.experimental import AnnCollection
from scdataset import scDataset, Streaming, BlockShuffling

## 1. Load the Tahoe-100M Dataset (h5ad)
We will use the AnnData library to load one or more Tahoe-100M h5ad files and combine them into an AnnCollection for efficient access.

In [ ]:
# Load multiple Tahoe-100M h5ad files and create an AnnCollection
h5ad_FILES_PATH = '/path/to/h5ad_folder'  # Update this to your local folder with the h5ad files
h5ad_paths = [f'{h5ad_FILES_PATH}/plate{i}_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad' for i in range(1, 2)] # Loading only 1 plate for demonstration

adatas = [ad.read_h5ad(path, backed='r') for path in h5ad_paths]
for adata in adatas:
    # Keep 'cell_line' column
    adata.obs = adata.obs[['cell_line']]
collection = AnnCollection(adatas)
print('Total cells:', collection.n_obs)
print('Total genes:', collection.n_vars)

Total cells: 5481420
Total genes: 62710


## 2. Cell Line Recognition Task and Stratified Split
For this tutorial, we will train a linear classifier to recognize the cell line of each cell. We will split the dataset into train and test sets using a stratified split on the `cell_line` column.

In [ ]:
# Get cell_line labels from the AnnCollection
cell_lines = collection.obs['cell_line'].values
indices = np.arange(collection.n_obs)

# Stratified train/test split on cell_line
train_idx, test_idx = train_test_split(
    indices, stratify=cell_lines, test_size=0.2, random_state=42)

print(f'Train size: {len(train_idx)}, Test size: {len(test_idx)}')

Train size: 4385136, Test size: 1096284


### Custom fetch_transform and batch_transform for AnnCollection

Before we wrap our data with scDataset, it's important to know that scDataset v0.2.0 allows you to provide custom functions for handling how data is fetched and transformed. This makes it easy to adapt to different data formats and preprocessing needs, especially for large or complex datasets like Tahoe-100M.

- **fetch_transform:** Ensures that each chunk from the AnnCollection is materialized as an in-memory AnnData object. This is necessary because AnnCollection chunks are lazy by default and do not load the full X matrix until `to_adata()` is called.
- **batch_transform:** Converts each AnnData batch into a tuple `(X, y)` suitable for PyTorch training. It densifies the X matrix if it is sparse and encodes the cell line labels as integer tensors. This makes each batch ready for direct use in a PyTorch model.

In [ ]:
# Prepare label encoder for cell lines
cell_line_encoder = LabelEncoder()
cell_line_encoder.fit(collection.obs['cell_line'].values)

# Define fetch_transform and batch_transform for scDataset
def fetch_transform(batch):
    # Materialize the AnnData batch (X matrix) in memory
    return batch.to_adata()

def batch_transform(batch, cell_line_encoder=cell_line_encoder):
    # Convert AnnData batch to (X, y) tensors for training
    X = batch.X.astype('float32')
    # Densify if X is a sparse matrix
    if sparse.issparse(X):
        X = X.toarray()
    X = torch.from_numpy(X)
    y = cell_line_encoder.transform(batch.obs['cell_line'].values)
    y = torch.from_numpy(y).long()
    return X, y

## 3. Wrap the AnnCollection with scDataset
To efficiently train and evaluate models, we use `scDataset` v0.2.0 with sampling strategies to wrap the AnnCollection and create PyTorch DataLoaders for both train and test splits.

**New in v0.2.0:** scDataset now uses a strategy-based approach for sampling. Each dataset requires a sampling strategy that defines how data is accessed and in what order.

For **training**, we'll use `BlockShuffling` to randomize data while maintaining some locality for better I/O performance.
For **evaluation**, we'll use `Streaming` for deterministic, sequential access.

You can tune `batch_size` and `fetch_factor` for your hardware; for best performance, set `prefetch_factor = fetch_factor + 1` in the DataLoader. The sampling strategies control how indices are generated and whether shuffling occurs. See the scDataset documentation for more details.

In [ ]:
# Set up scDataset for train and test splits
batch_size = 64
fetch_factor = 16
num_workers = 12

# Training split with block shuffling for randomization
train_strategy = BlockShuffling(block_size=8, indices=train_idx)
scdata_train = scDataset(
    data_collection=collection,
    strategy=train_strategy,
    batch_size=batch_size,
    fetch_factor=fetch_factor,
    fetch_transform=fetch_transform,
    batch_transform=batch_transform,
)

train_loader = DataLoader(
    scdata_train,
    batch_size=None,
    num_workers=num_workers,
    prefetch_factor=fetch_factor+1,
    persistent_workers=True,
    pin_memory=True
)

# Test split with streaming for deterministic evaluation
test_strategy = Streaming(indices=test_idx)
scdata_test = scDataset(
    data_collection=collection,
    strategy=test_strategy,
    batch_size=batch_size,
    fetch_factor=fetch_factor,
    fetch_transform=fetch_transform,
    batch_transform=batch_transform,
)

test_loader = DataLoader(
    scdata_test,
    batch_size=None,
    num_workers=num_workers,
    prefetch_factor=fetch_factor+1,
    persistent_workers=True,
    pin_memory=True
)

## 4. Train and Evaluate a Linear Classifier for Cell Line Recognition
We will train a simple linear classifier to predict the cell line from gene expression. The model will be trained for one epoch on the training set and evaluated on the test set.

In [ ]:
# Model definition
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_genes = collection.n_vars
num_classes = len(cell_line_encoder.classes_)
model = nn.Linear(num_genes, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Training loop (1 epoch)
model.train()
for i, (x, y) in enumerate(tqdm(train_loader, desc='Training')):
    x = x.to(device)
    y = y.to(device)
    logits = model(x)
    loss = loss_fn(logits, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 1000 == 0:
        print(f'Train batch {i}: loss = {loss.item():.4f}')

# Evaluation loop (one pass over test set)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x, y in tqdm(test_loader, desc='Evaluating'):
        x = x.to(device)
        y = y.to(device)
        logits = model(x)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
print(f'Test accuracy: {100.0 * correct / total:.2f}% ({correct}/{total})')

Training:   0%|          | 7/68512 [00:04<9:25:04,  2.02it/s] 

Train batch 0: loss = 3.9899


Training:   2%|▏         | 1050/68512 [00:22<19:27, 57.77it/s]

Train batch 1000: loss = 0.3440


Training:   3%|▎         | 2042/68512 [00:40<13:59, 79.20it/s] 

...

Train batch 67000: loss = 0.0083


Training:  99%|█████████▉| 68060/68512 [20:45<00:07, 59.74it/s] 

Train batch 68000: loss = 0.9251


Evaluating: 100%|██████████| 17130/17130 [05:17<00:00, 53.97it/s] 

Test accuracy: 97.11% (1064575/1096284)


## 5. Summary and Next Steps

You have now seen how to load the Tahoe-100M dataset, perform a stratified split, and train/test a linear classifier for cell line recognition using scDataset v0.2.0. 

**Key features demonstrated:**
- **Strategy-based sampling**: Used `BlockShuffling` for training randomization and `Streaming` for deterministic evaluation
- **Custom transforms**: Applied `fetch_transform` and `batch_transform` to handle AnnCollection data format
- **Efficient data loading**: Configured `fetch_factor` and multiprocessing for optimal performance

**Try exploring other sampling strategies:**
- `ClassBalancedSampling` for handling imbalanced cell type distributions
- `BlockWeightedSampling` for custom sample weighting
- `MultiIndexable` for handling multi-modal data (gene expression + protein data)

For more advanced usage and complete API documentation, see the scDataset source code.